In [1]:
%matplotlib 

Using matplotlib backend: <object object at 0x000001CBAC0190F0>


In [2]:
import torch

In [3]:
import torch.nn as nn

In [4]:
import torch.utils.data as data

In [5]:
import torchvision

In [6]:
import matplotlib.pyplot as pyp

In [7]:
import torchvision.transforms as trans

In [8]:
import numpy as np

In [9]:
import matplotlib.animation as animation

In [10]:
import scipy

In [11]:
from torch.nn import functional as F

In [12]:
transformCom=trans.Compose([trans.Resize(224),trans.ToTensor()])

In [13]:
trainSet=torchvision.datasets.FashionMNIST('../',train=True,download=True,transform=transformCom)
testSet=torchvision.datasets.FashionMNIST('../',train=False,download=True,transform=transformCom)

In [14]:
trainLoader=data.DataLoader(trainSet,batch_size=128,shuffle=True)
testLoader=data.DataLoader(testSet,batch_size=128,shuffle=True)

In [15]:
class Residual(nn.Module):
    def __init__(self, in_channels, out_channels,use_1X1=False ,strides=1):
        super(Residual, self).__init__()
        self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1,stride=strides);
        self.conv2=nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1,stride=1)
        if use_1X1 ==True:
            self.conv3=nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=strides)
        else:
            self.conv3=None;
        self.bn1=nn.BatchNorm2d(out_channels)
        self.bn2=nn.BatchNorm2d(out_channels)
        self.relu=nn.ReLU(inplace=True)

    def forward(self, X):
        Y=self.relu(self.bn1(self.conv1(X)))
        Y=self.bn2(self.conv2(Y))
        if self.conv3:
            X=self.conv3(X)
        Y+=X
        return self.relu(Y)
def ResidualBlock(in_channels, out_channels,num_res,first_block=False):
    blk=[]
    for i in range(num_res):
        if i==0 and not first_block:
            blk.append(Residual(in_channels,out_channels,use_1X1=True,strides=2))
        else:
            blk.append(Residual(out_channels,out_channels))
    return blk
b1=nn.Sequential(nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3),nn.BatchNorm2d(64),nn.ReLU(),nn.MaxPool2d(kernel_size=3,stride=2,padding=1))
b2=nn.Sequential(*ResidualBlock(64,64,2,first_block=True))
b3=nn.Sequential(*ResidualBlock(64,128,2))
b4=nn.Sequential(*ResidualBlock(128,256,2))
b5=nn.Sequential(*ResidualBlock(256,512,2))
ResNet=nn.Sequential(b1,b2,b3,b4,b5,nn.AdaptiveAvgPool2d((1,1)),nn.Flatten(),nn.Linear(512,10))
    
        
#ResNet = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))

In [16]:
# X = torch.rand(size=(1, 1, 96, 96))
# for layer in GoogLeNet:
#     X = layer(X)
#     print(layer.__class__.__name__, 'output shape:\t', X.shape)

In [17]:
if torch.cuda.is_available():
     ResNet.to(torch.device("cuda:0"))

In [18]:
optim=torch.optim.SGD(ResNet.parameters(),lr=0.2)
loss=nn.CrossEntropyLoss()

In [19]:
#创建曲线图并初始化
%matplotlib
xdata=[]
ydata=[]
xdataTest=[]
ydataTest=[]
xAccurate=[]
yAccurate=[]
xAccurateTest=[]
yAccurateTest=[]
fig, ax = pyp.subplots()
line, = ax.plot(xdata, ydata,color='blue')
line2, = ax.plot(xdataTest, ydataTest,color='red')
line3, = ax.plot(xAccurate, yAccurate,color='black')
line4, = ax.plot(xAccurateTest, yAccurateTest,color='green')
pyp.title("Loss")
pyp.xlabel("epoch")
pyp.ylabel("Loss")
pyp.grid()
ax.legend((line,line2,line3,line4),('trainLoss','testLoss','trainAccurate','testAccurate'))
#动态更新曲线训练函数
num_epoch=22
aveLoss=torch.zeros(num_epoch)
aveLoss=aveLoss.detach().numpy()
aveLossTest=torch.zeros(num_epoch)
aveLossTest=aveLossTest.detach().numpy()
aveaccurate=torch.zeros(num_epoch)
aveaccurate=aveaccurate.detach().numpy()
aveaccurateTest=torch.zeros(num_epoch)
aveaccurateTest=aveaccurateTest.detach().numpy()
#初始化参数为均匀分布，教训：初始化参数对训练是否收敛影响巨大，初始化为0完全不收敛   
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)
ResNet.apply(init_weights)

def animationTrain(epoch):
    #每epoch训练代码区#
    TotalAccuracy=0
    TotalAccuracyTest=0
    sumLoss=0
    trainnum=0;
    if epoch>0:
        for trainData,trainLabel in trainLoader:
            optim.zero_grad()
            Y=trainLabel.to(torch.device("cuda:0"))
            X=trainData.to(torch.device("cuda:0"))
            YHAT=ResNet(X)
            comLoss=loss(YHAT,Y)
            comLoss.backward()
            optim.step()
            sumLoss=comLoss+sumLoss
            trainnum=trainnum+1
            #训练集分类准确率计算#
            AccurateNum = (YHAT.argmax(1) == Y).sum()
            TotalAccuracy = TotalAccuracy + AccurateNum

            
        aveLoss[epoch-1]=(sumLoss.to(torch.device("cpu"))/trainnum)
        aveaccurate[epoch-1]=TotalAccuracy/len(trainSet)
#         print(aveLoss[epoch-1])
        #测试代码区
        sumLoss=0
        trainnum=0     
        with torch.no_grad():
            for trainData,trainLabel in testLoader:
                Y=trainLabel.to(torch.device("cuda:0"))
                X=trainData.to(torch.device("cuda:0"))
                YHAT=ResNet(X)
                comLoss=loss(YHAT,Y)
                sumLoss=comLoss+sumLoss
                trainnum=trainnum+1
                #测试集分类准确率计算
                AccurateNum = (YHAT.argmax(1) == Y).sum()
                TotalAccuracyTest = TotalAccuracyTest + AccurateNum
                         
        aveLossTest[epoch-1]=(sumLoss.to(torch.device("cpu"))/trainnum) 
        aveaccurateTest[epoch-1]=TotalAccuracyTest/len(testSet)
        print(aveLossTest[epoch-1])
        print( aveaccurateTest[epoch-1])
        #图表数据更新#
        xdata.append(epoch-1)
        ydata.append(aveLoss[epoch-1])
        xdataTest.append(epoch-1)
        ydataTest.append(aveLossTest[epoch-1])
        xAccurate.append(epoch-1)
        yAccurate.append(aveaccurate[epoch-1])
        xAccurateTest.append(epoch-1)
        yAccurateTest.append(aveaccurateTest[epoch-1])
        line.set_xdata(xdata)
        line.set_ydata(ydata)
        line2.set_xdata(xdataTest)
        line2.set_ydata(ydataTest)
        line3.set_xdata(xAccurate)
        line3.set_ydata(yAccurate)
        line4.set_xdata(xAccurateTest)
        line4.set_ydata(yAccurateTest)
        ax.set_xlim(0, max([max(xdata),max(xdataTest),max(xAccurate),max(xAccurateTest)]))
        ax.set_ylim(0, max([max(ydata),max(ydataTest),max(yAccurate),max(yAccurateTest)]))
        #print(aveLoss[epoch])
        return line,line2, line3,line4
#创建动画对象并显示，显示过程逐次调用Trian函数
anim = animation.FuncAnimation(fig, animationTrain, interval=20, blit=False,repeat=False,frames=num_epoch)
pyp.show()     

Using matplotlib backend: TkAgg
0.36144635
0.86289996


C:\Users\xyr\AppData\Local\Temp\ipykernel_11300\1477163302.py:98: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  ax.set_xlim(0, max([max(xdata),max(xdataTest),max(xAccurate),max(xAccurateTest)]))


0.26780027
0.90239996
0.22155088
0.91859996
0.23660894
0.9148
0.23384386
0.92109996
0.24824013
0.9165
0.25324845
0.9223
0.23213317
0.92499995
0.30265948
0.9159
0.26561806
0.9238
0.28601703
0.9273
0.30739155
0.92429996
0.32133767
0.9285
0.3353613
0.9237
0.3502507
0.9277
0.33464885
0.9284
0.32315543
0.93189996
0.3177604
0.93399996
0.3237472
0.93439996
0.3229851
0.9346
0.33292413
0.93509996
